In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import PIL
import json
from urbanComponent import *

from persim import plot_diagrams
from ripser import ripser, lower_star_img

c:\Users\Gustavo\anaconda3\envs\urbantk\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### Binning 3D data

In [2]:
# # Generate a random 3D array with values between 0 and 1
# arr = np.random.rand(5, 5, 5)

# x_extension = 300
# y_extension = 300
# z_extension = 300

# bin_size = 10

# # Define the number of bins for each dimension
# num_bins = [int(x_extension/bin_size), int(y_extension/bin_size), int(z_extension/bin_size)]

# # Compute the bin edges for each dimension
# bin_edges = [np.linspace(0, 300, n + 1) for n in num_bins]

# # Bin the array along each dimension
# binned = [np.digitize(arr[:, :, i], bin_edges[i]) for i in range(arr.shape[2])]

# # # Stack the binned arrays into a single n-dimensional array
# # binned_arr = np.stack(binned, axis=-1)

### Load joined data as dataframe with latitude, longitude, height, value

In [3]:
f = open('../../../public/data/manhattan_wo_cells/buildings_s_joined.json', 'r')

data = json.load(f)

joinedLayerIndex = 0

values = []

for joinedObject in data['joinedObjects']:
    if(joinedObject['joinedLayerIndex'] == joinedLayerIndex):
        values = joinedObject['otherValues']
        break

f.close()

uc = UrbanComponent()

uc.setWorkDir("../../../public/data/manhattan_wo_cells")
uc.addLayerFromJsonFile("../../../public/data/manhattan_wo_cells/buildings_s.json")

buildingsJson = uc.layers['json'][0]

# f = open('./buildings.json', 'w')

# f.write(json.dumps(buildingsJson))

# f.close()

latitude = []
longitude = []
height = []

for geometry in buildingsJson['data']:
    for i in range(int(len(geometry['geometry']['coordinates'])/3)):
        latitude.append(geometry['geometry']['coordinates'][i*3])
        longitude.append(geometry['geometry']['coordinates'][i*3+1])
        height.append(geometry['geometry']['coordinates'][i*3+2])

df = pd.DataFrame(data={'latitude': latitude, 'longitude': longitude, 'height': height})

m1 = df[['latitude','longitude','height']].to_numpy()[:,1]
m2 = df[['latitude','longitude','height']].to_numpy()[:,0]
m3 = df[['latitude','longitude','height']].to_numpy()[:,2]

xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()
zmin = m3.min()
zmax = m3.max()

X, Y, Z = np.mgrid[xmin:xmax:100, ymin:ymax:100, zmin:zmax:100]





### Creating 3D scalar function with KDE

In [ ]:
# df = pd.read_pickle('data/311_2019.pkl.gz')
# m1 = df[['latitude','longitude']].to_numpy()[:,1]
# m2 = df[['latitude','longitude']].to_numpy()[:,0]
# xmin = m1.min()
# xmax = m1.max()
# ymin = m2.min()
# ymax = m2.max()
# X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
# positions = np.vstack([X.ravel(), Y.ravel()])
# values = np.vstack([m1, m2])
# kernel = scipy.stats.gaussian_kde(values)
# Z = np.reshape(kernel(positions).T, X.shape)

### Plotting scalar function

In [ ]:
# fig, ax = plt.subplots()
# ax.imshow(np.rot90(Z), cmap=plt.cm.YlOrBr, extent=[xmin, xmax, ymin, ymax])
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
# plt.show()